Load the environment variables

In [14]:
from dotenv import load_dotenv

load_dotenv()

True

Marking the GraphState

In [15]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

Custom Tools

Tools are interfaces that an agent can use to interact with the world. They combine a few things:
1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user

In [16]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
    """Returns the input as all lower case."""
    return input.lower()

@tool("upper_case", return_direct=True)
def to_upper_case(input:str) -> str:
    """Returns the input as all upper case."""
    return input.upper()

@tool("random_number", return_direct=True)
def generate_random_number(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case, to_upper_case, generate_random_number]

#generate_random_numner.run('random')
#to_lower_case.run('SaNdEeP')
#to_upper_case.run('sAnDeEp')


Agent - with new create_open_ai

In [17]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
'''
inputs = {
    "input": "Give me a random number and then write it in words and then convert them into both lower and upper case.",
    "chat_history": [],
    "intermediate_steps": [],
}

agent_outcome = agent_runnable.invoke(inputs)

agent_outcome
'''

Nodes

In [18]:
# Tools Executor
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [19]:
# Define the agent / graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is a the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `end` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"


Define the graph

In [20]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally, we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that adter `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


In [22]:
inputs = {
    "input": "Give me a random number and then write it in words and then convert them into lower case.",
    "chat_history": [],
    "intermediate_steps": [],
}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("-----")

{'agent_outcome': AgentActionMessageLog(tool='random_number', tool_input={'input': 'generate random number'}, log="\nInvoking: `random_number` with `{'input': 'generate random number'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"generate random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-c7a2289d-a8fc-466d-8e8a-519da35d198c-0')])}
-----
The agent action is tool='random_number' tool_input={'input': 'generate random number'} log="\nInvoking: `random_number` with `{'input': 'generate random number'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"generate random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-c7a2289d-a8fc-466d-8e8a-519da35d198c-0')]
The tool result is: 12
{'intermediate_steps': [(AgentActionMessageLog(tool='random_number', tool_input={'input': 'gener

In [23]:
inputs = {
    "input": "Give me a random number and then write it in words and then convert it into lower case.",
    "chat_history": [],
    "intermediate_steps": [],
}

output = app.invoke(inputs)


The agent action is tool='random_number' tool_input={'input': 'random'} log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-a75693d0-84a9-4b8f-9a89-d954c6033d70-0')]
The tool result is: 0
The agent action is tool='lower_case' tool_input={'input': 'ZERO'} log="\nInvoking: `lower_case` with `{'input': 'ZERO'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"ZERO"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call'}, id='run-d6d081a5-9594-4a46-b100-cbade392626e-0')]
The tool result is: zero


In [25]:
inputs = {
    "input": "Does it get cold in Bangalore during April to June?",
    "chat_history": [],
    "intermediate_steps": [],
}

output = app.invoke(inputs)

output.get("agent_outcome").return_values['output']

"Bangalore generally experiences warm weather during April to June, with temperatures ranging from 24°C to 35°C. While it can get warm during the day, the evenings can be relatively cooler. It's always a good idea to check the current weather conditions closer to your travel dates for the most accurate information."